In [2]:
import os
import pickle
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from openai import OpenAI

def update_google_sheet(service, spreadsheet_id, sheet_name, titles, descriptions, scores):
    # Consolidate updates in a batch request
    batch_update_data = [
        {"range": f"{sheet_name}!B2:B", "values": [[title] for title in titles]},
        {"range": f"{sheet_name}!C2:C", "values": [[description] for description in descriptions]},
        {"range": f"{sheet_name}!H2:H", "values": [[score] for score in scores]}
    ]

    service.spreadsheets().values().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body={"valueInputOption": "USER_ENTERED", "data": batch_update_data}
    ).execute()

def generate_captions_and_update_sheet(service, spreadsheet_id, sheet_name, client):
    # Fetch descriptions
    descriptions = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id, range=f"{sheet_name}!C2:C"
    ).execute().get('values', [])

    # Concatenate all the text in column E
    entire_column_E = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id, range=f"{sheet_name}!E2:E"
    ).execute().get('values', [])
    concatenated_text = ' '.join([cell[0] for cell in entire_column_E if cell])

    # Generate captions
    captions = []
    for i, description in enumerate(descriptions):
        try:
            if description:
                prompt = f"Write a caption about the following around 100 words, starting with one emoticon: {description[0]}"
                response = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[{"role": "system", "content": "You are a creative assistant."},
                              {"role": "user", "content": prompt}]
                )
                caption = response.choices[0].message.content.strip() + " " + concatenated_text
                captions.append([caption])
            else:
                captions.append([''])
        except Exception as e:
            captions.append([''])
            print(f"Error processing description at row {i+2}: {e}")

    # Update the sheet with the new captions
    if captions:
        caption_range = f"{sheet_name}!D2:D{1+len(captions)}"
        service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id, range=caption_range,
            valueInputOption="USER_ENTERED", body={'values': captions}
        ).execute()

# Function to fetch video titles and links from Google Drive
def list_drive_videos(service, folder_id):
    try:
        # Query to access the shared folder and list video files
        results = service.files().list(
            supportsAllDrives=True, 
            includeItemsFromAllDrives=True, 
            q=f"parents in '{folder_id}' and mimeType='video/mp4' and trashed = false", 
            fields="nextPageToken, files(id, name)"
        ).execute()

        # Creating a list of videos with titles and links
        video_list = [{'title': file['name'], 'link': f"https://drive.google.com/file/d/{file['id']}/view"} for file in results.get('files', [])]
        return video_list

    except Exception as e:
        print(f"Error accessing Google Drive: {e}")
        return []

# Function to normalize titles for matching
def normalize_title(title):
    # Trim, replace underscores and colons with spaces, and remove file extension
    return title.strip().replace("_", " ").replace(":", " ").replace(".mp4", "").lower()

# Function to match and update Sheet with Google Drive links
def match_and_update_sheet(sheet_service, drive_service, spreadsheet_id, sheet_name, folder_id):
    drive_videos = list_drive_videos(drive_service, folder_id)
    sheet_titles = sheet_service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id, range=f"{sheet_name}!B2:B"
    ).execute().get('values', [])

    links = []
    for sheet_title in sheet_titles:
        if sheet_title:
            normalized_sheet_title = normalize_title(sheet_title[0])
            for video in drive_videos:
                normalized_video_title = normalize_title(video['title'])
                if normalized_video_title.lower() == normalized_sheet_title.lower():
                    matched_video = video
                    break
            else:
                matched_video = None

            print(f"Sheet Title: '{normalized_sheet_title}', Video Title: '{normalized_video_title if matched_video else 'N/A'}', Matched Video Link: {matched_video['link'] if matched_video else 'No match found'}")

            links.append([matched_video['link'] if matched_video else ''])

    # Update the sheet with the new links
    if links:
        links_range = f"{sheet_name}!A2:A{len(links)+1}"
        sheet_service.spreadsheets().values().update(
            spreadsheetId=spreadsheet_id, range=links_range,
            valueInputOption="USER_ENTERED", body={'values': links}
        ).execute()

# Setup ChromeDriver
chrome_driver_path = 'C:\\Users\\David\\Downloads\\chromedriver-win32\\chromedriver-win32\\chromedriver.exe'
service = Service(executable_path=chrome_driver_path)
driver = webdriver.Chrome(service=service)

# Define scopes for Google Sheets and Drive
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# Define file paths
CLIENT_SECRET_FILE = 'C:\\Users\\David\\Downloads\\credentials.json'
TOKEN_PICKLE_FILE = 'token.pickle'

# Get credentials
def get_credentials():
    creds = None
    if os.path.exists(TOKEN_PICKLE_FILE):
        with open(TOKEN_PICKLE_FILE, 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        with open(TOKEN_PICKLE_FILE, 'wb') as token:
            pickle.dump(creds, token)
    return creds

creds = get_credentials()

# Setup Google Sheets and Drive API Clients
sheet_service = build('sheets', 'v4', credentials=creds)
drive_service = build('drive', 'v3', credentials=creds)

# Spreadsheet ID and Sheet Name
spreadsheet_id = '1tdwufrfmL0Bc2lnehOEopSo9T9w717LvbsjNhT8xqrE'
sheet_name = "Copy of Tim Draper"

# Setup OpenAI API Client
openai_client = OpenAI()

try:
    driver.get("https://clip.opus.pro/clip/P0123005tWpp")
    wait = WebDriverWait(driver, 20)
    wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, ".MuiTypography-root.MuiTypography-base.css-1wc18uy")))

    titles = driver.find_elements(By.CSS_SELECTOR, ".MuiTypography-root.MuiTypography-xl2.css-1jfjns0")
    descriptions = driver.find_elements(By.CSS_SELECTOR, ".MuiTypography-root.MuiTypography-base.css-1wc18uy")
    scores = driver.find_elements(By.CSS_SELECTOR, ".MuiTypography-root.MuiTypography-xl3.css-tz3tqc")

    titles_list, descriptions_list, scores_list = [], [], []
    for title, description, score in zip(titles, descriptions, scores):
        score_value = int(score.text)
        if score_value > 80:
            titles_list.append(title.text)
            descriptions_list.append(description.text)
            scores_list.append(str(score_value))

    # Sort by scores and update the sheet
    combined = sorted(zip(titles_list, descriptions_list, scores_list), key=lambda x: int(x[2]), reverse=True)
    sorted_titles, sorted_descriptions, sorted_scores = zip(*combined)

    update_google_sheet(sheet_service, spreadsheet_id, sheet_name, sorted_titles, sorted_descriptions, sorted_scores)
    generate_captions_and_update_sheet(sheet_service, spreadsheet_id, sheet_name, openai_client)

    # Fetch videos from Google Drive and update the sheet
    match_and_update_sheet(sheet_service, drive_service, spreadsheet_id, sheet_name, '1qMOgHJNSSv1fT_AsKo14Ix4rAPn9-Xo-')

except Exception as e:
    print(f"An error occurred: {str(e)}")

finally:
    driver.quit()

Sheet Title: 'revolutionizing education  draper university unleashes creativity and breaks free from accreditation', Video Title: 'revolutionizing education  draper university unleashes creativity and breaks free from accreditation', Matched Video Link: https://drive.google.com/file/d/1EyG17d1T3psvECfwzX5qXsO3vyHQRYmx/view
Sheet Title: 'unlocking the power of math  how lemonade stands inspire kids', Video Title: 'unlocking the power of math  how lemonade stands inspire kids', Matched Video Link: https://drive.google.com/file/d/1aRMC32D73wyB801J64M6WxR1ZBKPCM2d/view
Sheet Title: 'unleashing generational fire  investing, wealth, and knowledge', Video Title: 'unleashing generational fire  investing, wealth, and knowledge', Matched Video Link: https://drive.google.com/file/d/1ONVYIViXLjD1NbavB-5iAWSK-Swqq9BD/view
Sheet Title: 'from million to billion  the explosive success of an investment pro', Video Title: 'from million to billion  the explosive success of an investment pro', Matched Vid